In [3]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [4]:
import dotenv

dotenv.load_dotenv()

True

In [7]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()
location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1157270
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "sweden", "city": "stockholm", "street": "stockholm-st-eriksgatan-83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": 59.33, "longitude": 18.07}'

In [8]:
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [11]:
aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df


,pm25,country,city,street,date,url
0,12.0,sweden,stockholm,stockholm-st-eriksgatan-83,2024-11-13,https://api.waqi.info/feed/@10523


In [13]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


In [16]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# Gets forecast for next 10 days
# Replace  hourly forecasts with a single daily forecast
# Get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])     #To date format

daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-13,4.75,0.0,14.799459,228.945282,stockholm
1,2024-11-14,5.80,0.0,15.021105,315.970917,stockholm
2,2024-11-15,6.80,0.0,9.064569,276.842682,stockholm
3,2024-11-16,10.80,0.0,26.260265,241.326950,stockholm
4,2024-11-17,5.00,0.6,20.371038,223.567963,stockholm
5,2024-11-18,4.60,0.0,14.044615,268.531219,stockholm
6,2024-11-19,1.45,0.0,4.334974,311.633453,stockholm
7,2024-11-20,3.30,1.4,23.402769,89.118622,stockholm
8,2024-11-21,6.90,1.1,11.367109,259.045990,stockholm
9,2024-11-22,1.50,1.8,30.302078,256.954346,stockholm


In [17]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


In [18]:
air_quality_fg.insert(aq_today_df)  #insert air quality to feature store

2024-11-13 09:42:03,724 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157270/fs/1147973/fg/1345959


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157270/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x107707e20>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669751
         }
       },
       "result": {
         "observed_value": 12.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T08:42:03.000722Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "unsucc

In [19]:
weather_fg.insert(daily_df)     #Insert weather data to feature store

2024-11-13 09:44:03,124 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1157270/fs/1147973/fg/1345960


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1157270/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17dbf9660>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 669752
         }
       },
       "result": {
         "observed_value": 4.33497428894043,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-13T08:44:03.000124Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min